<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao34bis_RAG_FlanT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Exemplo de RAG (Retrieval-Augmented Generation)

## ✨ VERSÃO SEM API EXTERNA (100% LOCAL NO COLAB) ✨

Este notebook demonstra um fluxo RAG simples usando:
 1. `sentence-transformers` (para Embeddings/Retrieval)
 2. `faiss-cpu` (para o Índice Vetorial)
 3. `transformers` (Hugging Face) com `google/flan-t5-base` (para Geração)

In [1]:
# (Adicionamos 'transformers' e 'accelerate')
!pip install -q -U sentence-transformers faiss-cpu transformers accelerate

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import textwrap

print("Bibliotecas instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.3 MB/s eta 0:00:00
Bibliotecas instaladas com sucesso!


In [2]:
# Esta é a nossa pequena "base de dados" de onde o RAG vai "puxar" informações.
corpus = [
    "O sol é uma estrela localizada no centro do Sistema Solar.",
    "A fotossíntese é o processo pelo qual as plantas usam a luz solar para produzir alimentos.",
    "O oceano Atlântico é o segundo maior oceano do mundo.",
    "A Torre Eiffel, localizada em Paris, foi concluída em 1889.",
    "A Amazônia é a maior floresta tropical do mundo e abriga milhões de espécies.",
    "O ciclo da água descreve o movimento contínuo da água na Terra, através da evaporação, condensação e precipitação."
]

print(f"Corpus de conhecimento com {len(corpus)} documentos.")

Corpus de conhecimento com 6 documentos.


In [3]:
# Usamos 'sentence-transformers' para converter nossos textos em vetores numéricos.
print("Carregando modelo de embeddings (SentenceTransformer)...")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Modelo de embeddings carregado.")

Carregando modelo de embeddings (SentenceTransformer)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo de embeddings carregado.


In [4]:
# 1. Gerar embeddings para todo o nosso corpus
print("Gerando embeddings para o corpus...")
corpus_embeddings = embedding_model.encode(corpus)

# 2. Definir a dimensão dos vetores
d = corpus_embeddings.shape[1]

# 3. Criar um índice FAISS
print("Criando índice FAISS...")
index = faiss.IndexFlatL2(d)

# 4. Adicionar os vetores ao índice
index.add(corpus_embeddings)
print(f"Índice FAISS criado e {index.ntotal} vetores adicionados.")

Gerando embeddings para o corpus...
Criando índice FAISS...
Índice FAISS criado e 6 vetores adicionados.


In [5]:
# --- Esta é a principal mudança ---
# Usaremos o 'google/flan-t5-base' da Hugging Face
print("Carregando modelo generativo (Flan-T5)...")

# Detecta se há GPU disponível (fundamental para velocidade)
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

print(f"Modelo Flan-T5 carregado e rodando em: {device.upper()}")

Carregando modelo generativo (Flan-T5)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Modelo Flan-T5 carregado e rodando em: CPU


In [6]:
def executar_consulta_rag_local(pergunta, k=2):
    """
    Executa o fluxo RAG completo com o modelo local.
    """
    print(f"--- Nova Consulta RAG ---")
    print(f"Pergunta original: {pergunta}\n")

    # --- 1. RETRIEVAL ---
    # (Igual ao notebook anterior)
    pergunta_embedding = embedding_model.encode([pergunta])
    D, I = index.search(pergunta_embedding, k)
    documentos_relevantes = [corpus[i] for i in I[0]]

    print(f"Documentos recuperados (Contexto):")
    for i, doc in enumerate(documentos_relevantes):
        print(f"  {i+1}. {doc}")
    print("\n")

    # --- 2. AUGMENTATION ---
    # Criamos um prompt específico para modelos instruction-tuned (como o Flan-T5)
    contexto = "\n".join(documentos_relevantes)

    # O Flan-T5 funciona bem com prompts no formato "Contexto: ... Pergunta: ... Resposta:"
    prompt_aumentado = f"""
    Contexto:
    {contexto}

    Pergunta:
    {pergunta}

    Com base no contexto, responda à pergunta:
    """

    # --- 3. GENERATION ---
    # (Esta parte muda para usar o 'transformers')
    print("Gerando resposta com o Flan-T5...")

    # 1. Tokenizar o prompt
    inputs = tokenizer(prompt_aumentado, return_tensors="pt", max_length=512, truncation=True).to(device)

    # 2. Gerar a resposta
    # max_new_tokens controla o tamanho da resposta
    outputs = llm_model.generate(**inputs, max_new_tokens=100)

    # 3. Decodificar a resposta
    resposta_texto = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return textwrap.fill(resposta_texto, width=80)

In [7]:
# --- Teste 1: Pergunta cuja resposta ESTÁ no corpus ---
pergunta1 = "O que é fotossíntese?"
resposta1 = executar_consulta_rag_local(pergunta1, k=1)

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta1)
print("-" * 30)

# --- Teste 2: Pergunta cuja resposta ESTÁ no corpus, mas é mais específica ---
pergunta2 = "Onde fica a Torre Eiffel e quando foi concluída?"
resposta2 = executar_consulta_rag_local(pergunta2, k=2)

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta2)
print("-" * 30)

# --- Teste 3: Pergunta cuja resposta NÃO ESTÁ no corpus ---
# O Flan-T5 é forçado a usar o contexto. Se não está lá, ele não "inventa".
pergunta3 = "Quem foi o primeiro presidente do Brasil?"
resposta3 = executar_consulta_rag_local(pergunta3, k=2)

print("\n--- RESPOSTA FINAL (RAG) ---")
print(resposta3)
print("-" * 30)

--- Nova Consulta RAG ---
Pergunta original: O que é fotossíntese?

Documentos recuperados (Contexto):
  1. A fotossíntese é o processo pelo qual as plantas usam a luz solar para produzir alimentos.


Gerando resposta com o Flan-T5...

--- RESPOSTA FINAL (RAG) ---
A fotossntese é o processo pelo qual as plantas usam a luz solar para produzir
alimentos.
------------------------------
--- Nova Consulta RAG ---
Pergunta original: Onde fica a Torre Eiffel e quando foi concluída?

Documentos recuperados (Contexto):
  1. A Torre Eiffel, localizada em Paris, foi concluída em 1889.
  2. O ciclo da água descreve o movimento contínuo da água na Terra, através da evaporação, condensação e precipitação.


Gerando resposta com o Flan-T5...

--- RESPOSTA FINAL (RAG) ---
Paris
------------------------------
--- Nova Consulta RAG ---
Pergunta original: Quem foi o primeiro presidente do Brasil?

Documentos recuperados (Contexto):
  1. A Amazônia é a maior floresta tropical do mundo e abriga milhões de 